In [ ]:

def sequence_list_length_by_1():

def dephaseing_kraus(p):
    Ad0 = np.asarray([[1, 0], [0, np.sqrt(1 - p)]])
    Ad1 = np.asarray([[0, np.sqrt(p)], [0, 0]])
    return [Ad0, Ad1]

def depolarizing_kraus(p):
    M0 = np.sqrt(1 - p) * I
    M1 = np.sqrt(p) * X
    return [M0, M1]
if __name__ == '__main__':
####### input
    rb_types=int(input('-1 rb, 0 decomp_rb, 1 prb\n'))
    max_seq= int(input('max seq \n'))
####### rb gate number i    
    for i in tqdm(sequence_length,ascii=True,desc="sequence_length",dynamic_ncols=True):
####### generate rb sequence        
        pool = Pool()
        seq = pool.map(sequence_list_length_by_1,[i]*types)
        pool.close()
        pool.join()
####### rb sequence into probablility
        types_probability=[]
        m_average=[]
        pool = mp.Pool()
        repeat_probability = pool.starmap(sequence_to_probability,[(i,None,d,e) for d in range(1) for e in seq ])
        pool.close()
        pool.join()